# Elastic Inference Video Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in images from a video.

# Imports

In [ ]:
import numpy as np
import cv2
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import time

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

## Env setup

In [ ]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "Faster RCNN with ResNet50" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [ ]:
# What model to download.
MODEL_NAME = 'faster_rcnn_resnet50_coco_2018_01_28'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

## Download Model

In [ ]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [ ]:
from tensorflow.contrib.ei.python.predictor.ei_predictor import EIPredictor


ei_predictor = EIPredictor(
                model_dir=PATH_TO_FROZEN_GRAPH,
                input_names={"inputs":"image_tensor:0"},
                output_names={"detections_scores":"detection_scores:0",
                              "detection_classes":"detection_classes:0",
                              "detection_boxes":"detection_boxes:0",
                              "num_detections":"num_detections:0"},
                use_ei=False)

print('Loaded normal predictor')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## Helper code

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [ ]:
VIDEO_PATH = "/models/research/object_detection/dog_park.mp4"

In [ ]:
def extract_video_frames(pathIn):
    frames = []
    cap = cv2.VideoCapture(pathIn)
    count = 0
 
    while (cap.isOpened()):
 
        # Capture frame-by-frame
        ret, frame = cap.read()
 
        # Infer over 
        if ret == True and count < 40:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
            count += 1
        else:
            break
 
    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()
    return frames


def run_inference_for_single_image(image, ei_predictor):
    
  t1 = time.time()
  output_dict = ei_predictor({"inputs": image})
  t2 = time.time()
  inference_time = t2-t1
    
  # all outputs are float32 numpy arrays, so convert types as appropriate
  output_dict['num_detections'] = int(output_dict['num_detections'][0])
  output_dict['detection_classes'] = output_dict[
      'detection_classes'][0].astype(np.uint8)
  output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
  output_dict['detection_scores'] = output_dict['detections_scores'][0]
  
  return output_dict, inference_time

In [ ]:
# Time model load
dummy_input = np.random.rand(1,100,100,3) #256,256,3)
t1 = time.time()
_dummy_output = ei_predictor({"inputs": dummy_input})
t2 = time.time()
model_load_time = t2-t1
print('Model load time (seconds): {}'.format(model_load_time))

annotated_video_frames = []
inference_times = []
for i, image in enumerate(extract_video_frames(VIDEO_PATH)):
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image, axis=0)
  # Actual detection.
  output_dict, inference_time = run_inference_for_single_image(image_np_expanded, ei_predictor)
  inference_times.append(inference_time)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  
  annotated_video_frames.append(image)

del(ei_predictor)
num_frames = len(inference_times)
total_inference_time = sum(inference_times)
print("Number of video frames: {}\nAverage inference time (seconds): {}\nTotal time taken (seconds): {}"
      .format(num_frames, total_inference_time/num_frames, total_inference_time))

In [ ]:
ei_predictor = EIPredictor(
                model_dir=PATH_TO_FROZEN_GRAPH,
                input_names={"inputs":"image_tensor:0"},
                output_names={"detections_scores":"detection_scores:0",
                              "detection_classes":"detection_classes:0",
                              "detection_boxes":"detection_boxes:0",
                              "num_detections":"num_detections:0"},
                use_ei=True)

# Time model load
dummy_input = np.random.rand(1,100,100,3) #256,256,3)
t1 = time.time()
_dummy_output = ei_predictor({"inputs": dummy_input})
t2 = time.time()
model_load_time = t2-t1
print('Model load time (seconds): {}'.format(model_load_time))

annotated_video_frames = []
inference_times = []
for i, image in enumerate(extract_video_frames(VIDEO_PATH)):
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image, axis=0)
  # Actual detection.
  output_dict, inference_time = run_inference_for_single_image(image_np_expanded, ei_predictor)
  inference_times.append(inference_time)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  
  annotated_video_frames.append(image)

del(image)
del(ei_predictor)
num_frames = len(inference_times)
total_inference_time = sum(inference_times)
print("Number of video frames: {}\nAverage inference time (seconds): {}\nTotal time taken (seconds): {}"
      .format(num_frames, total_inference_time/num_frames, total_inference_time))

In [ ]:
%pylab inline
import io
import base64
from IPython.display import HTML
from IPython.display import clear_output
from ipywidgets import DOMWidget, Image, Video

height,width,layers=annotated_video_frames[1].shape

video=cv2.VideoWriter('annotated_dog_park.avi',
                      cv2.VideoWriter_fourcc('M','J','P','G'),
                      60, (width,height))

for frame in annotated_video_frames:
    video.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    
del(frame)
del(annotated_video_frames)
cv2.destroyAllWindows()
video.release()


HTML("""
    <video controls src="{0}" loop=1 autoplay width=100%/> 
    """.format('dog_park.mp4')
)

In [ ]:
import ffmpeg

(
    ffmpeg
    .input('annotated_dog_park.avi')
    .output('annotated_dog_park.mp4')
    .overwrite_output()
    .run()
)

HTML("""
    <video controls src="{0}" loop=1 autoplay width=100%/> 
    """.format('annotated_dog_park.mp4')
)